# ProbLog as a Python library

ProbLog is mostly written in Python (except the knowledge compilation step, which is in C). This allows us to import ProbLog as a Python package. In this document we show some examples to illustrate this usage.

## Add ProbLog to your Python path

We first add the path to the ProbLog directory to the Python path such that it can be imported. If ProbLog is already in your path, this step can be skipped

In [1]:
import sys, os
sys.path.append(os.path.join('..','..'))
sys.path.append(os.path.abspath(os.path.join(os.environ['PROBLOGDIR'])))

## ProbLog as a string

A ProbLog program can be composed as a string and fed into ProbLog:

In [2]:
from problog.program import PrologString
from problog.core import ProbLog
from problog.sdd_formula import SDD

p = PrologString("""coin(c1). coin(c2).
0.4::heads(C); 0.6::tails(C) :- coin(C).
win :- heads(C).
query(win).
""")

print(ProbLog.convert(p, SDD).evaluate())

{win: 0.64}


In this example, we ask ProbLog to convert the ProbLog string to an SDD, which can be evaluated.

### Controlling the ProbLog chain

You can also control the intermediate steps in as much detail as you wish:

In [3]:
from problog.program import PrologString, PrologFile
from problog.core import ProbLog
from problog.logic import Term
from problog.evaluator import SemiringSymbolic, Evaluator
from problog.engine import DefaultEngine
from problog.nnf_formula import NNF
from problog.cnf_formula import CNF
from problog.sdd_formula import SDD

#p = PrologFile('path/to/myprogram.pl')
p = PrologString("""coin(c1). coin(c2).
0.4::heads(C); 0.6::tails(C) :- coin(C).
win :- heads(C).
query(win).
""")

# Prepare ProbLog engine
engine = DefaultEngine(label_all=True)
db = engine.prepare(p)

# Collect queries
queries = engine.query(db, Term( 'query', None ))
print ('Queries:', ', '.join([ str(q[0]) for q in queries ]))

# Colllect evidence
evidence = engine.query(db, Term( 'evidence', None, None ))
print ('Evidence:', ', '.join([ '%s=%s' % ev for ev in evidence ]))

# Ground program and make acyclic
gp = engine.ground_all(db)
gp = gp.makeAcyclic() # Not requried for SDDs

# Create NNF circuit from program
nnf = SDD.createFrom(gp) # Use SDDs
#nnf = NNF.createFrom(CNF.createFrom(gp)) # Use d-DNNFs

# Evaluate the circuit to obtain probabilities
result = nnf.evaluate()

print(result)

Queries: win
Evidence: 
{win: 0.64}


## ProbLog as Python datastructures

Instead of feeding a string and using the ProbLog syntax, it is also possible to create the program using Python datastructures:

In [4]:
from problog.program import SimpleProgram
from problog.logic import Constant,Var,Term,AnnotatedDisjunction

coin,heads,tails,win,query = Term('coin'),Term('heads'),Term('tails'),Term('win'),Term('query')
C = Var('C')
p = SimpleProgram()
p += coin(Constant('c1'))
p += coin(Constant('c2'))
p += AnnotatedDisjunction([heads(C,p=Constant(0.4)), tails(C,p=Constant(0.6))], coin(C))
p += (win << heads(C))
p += query(win)

print(ProbLog.convert(p, SDD).evaluate())

{win: 0.64}


The `<<` syntax is used to build a Prolog rule.

## Call Python definitions from ProbLog

It is also possible to call Python definition from ProbLog while grounding a program.

Python definitions can be made discoverable for ProbLog by using a `problog_extern` decorator. Suppose that you create a file `mylib.py` that contains the following Python code:

In [5]:
from problog.extern import problog_export

@problog_export('+str', '+str', '-str')
def concat_str(arg1, arg2):
    return arg1 + arg2


@problog_export('+int', '+int', '-int')
def int_plus(arg1, arg2):
    return arg1 + arg2


@problog_export('+list', '+list', '-list')
def concat_list(arg1, arg2):
    return arg1 + arg2

These functions are discoverable by ProbLog after using a `:- use_modele('mylib.py').` rule in ProbLog. Afterwards, we can use them as regular predicates:

In [6]:
p = PrologString("""
:- use_module('mylib.py').

query(concat_str(a,b,Z)).
query(concat_list([a,b],[c,d],Z)).
query(int_plus(1,2,Z)).
query(concat_list([a,b],[c],Y)).
""")

result = ProbLog.convert(p, SDD).evaluate()
for it in result.items() :
    print ('%s : %s' % (it))


int_plus(1,2,3) : 1.0
concat_str(a,b,ab) : 1.0
concat_list([a, b],[c, d],[a, b, c, d]) : 1.0
concat_list([a, b],[c],[a, b, c]) : 1.0


## Extra information

More information can be found on the ProbLog website:  
http://dtai.cs.kuleuven.be/problog

(c) 2015, KU Leuven, DTAI Research Group